In [1]:
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare Data

In [2]:
str_col = ['reference', 'translation']
num_col = ['ref_tox', 'trn_tox', 'similarity', 'lenght_diff']

data = pd.read_csv("data/interim/preprocessed_filtered_1.tsv", sep='\t', index_col=0)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 577777 entries, 0 to 577776
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   reference    577777 non-null  object 
 1   translation  577777 non-null  object 
 2   ref_tox      577777 non-null  float64
 3   trn_tox      577777 non-null  float64
 4   similarity   577777 non-null  float64
 5   length_diff  577777 non-null  float64
dtypes: float64(4), object(2)
memory usage: 30.9+ MB


# Prepare Dataloader

In [4]:
from src.data.make_dataset import prepareData, get_dataloader, tensorFromSentence

# Architecture of seq2seq model

In [5]:
from src.models.seq2seq import Seq2Seq, Encoder, Decoder

# Train model

In [6]:
from src.models.train_model import train

In [7]:
epochs = 10
embed_size = 128
hidden_size = 128
batch_size = 32
lr = 1e-3
MAX_LENGTH = 11
model_path = "seq2seq.pt"

vocab_tox, vocab_detox, pairs = prepareData(data, MAX_LENGTH)
train_dataloader, val_dataloader = get_dataloader(batch_size, vocab_tox, vocab_detox, pairs, MAX_LENGTH, device=device)

Counted words:
tox-vocab 27299
detox-vocab 22795


In [ ]:
encoder = Encoder(vocab_tox.n_words, embed_size, hidden_size, vocab_tox, device=device, max_length=MAX_LENGTH)
decoder = Decoder(embed_size, hidden_size, vocab_detox.n_words, vocab_detox, device=device, max_length=MAX_LENGTH)
seq2seq_model = Seq2Seq(encoder, decoder).to(device)

optimizer = optim.Adam(seq2seq_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(ignore_index=vocab_detox.word2index['<pad>'])

loss_train, loss_val = train(seq2seq_model, train_dataloader, val_dataloader, optimizer=optimizer, criterion=criterion, epochs=epochs, lr=lr, model_path=model_path)

Epoch 4: train:  88%|████████▊ | 4465/5067 [01:00<00:08, 74.97it/s, loss=2.08]

# Plot loss

In [ ]:
from src.visualization.visualize import plot_loss

plot_loss(loss_train, loss_val, epochs)

# Check in random sample from dataset

In [ ]:
def evaluate(model, sentence, vocab_tox, vocab_detox):
    with torch.no_grad():
        model.eval()
        input_tensor = tensorFromSentence(vocab_tox, sentence, device=device)

        outputs = model(input_tensor)

        _, topi = outputs.topk(1)
        ids = topi.squeeze()

        words = []
        for idx in ids:
            if idx.item() == vocab_tox.word2index['<eos>']:
                break
            words.append(vocab_detox.index2word[idx.item()])
    return words

In [ ]:
import random
import string

def evaluateRandomly(model, vocab_tox, vocab_detox, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('origin:     ', pair[0])
        print('translated: ', pair[1])
        output_words = evaluate(model, pair[0], vocab_tox, vocab_detox)
        output_sentence = "".join([" "+i if not i.startswith("'") and not i.startswith("n'") and i not in string.punctuation else i for i in output_words]).strip()
        print('predicted:  ', output_sentence)
        print('')

In [ ]:
load_seq2seq_model = torch.load(model_path)

load_seq2seq_model.eval()
evaluateRandomly(load_seq2seq_model, vocab_tox, vocab_detox)